#  Long short-term memory (LSTM) RNNs

In [51]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
import numpy as np
mx.random.seed(1)
ctx = mx.gpu(0)

## Dataset: "The Time Machine" 

In [52]:
with open("data/nlp/timemachine.txt") as f:
    time_machine = f.read()
time_machine = time_machine[:-38083]

## Numerical representations of characters

In [53]:
character_list = list(set(time_machine))
vocab_size = len(character_list)
character_dict = {}
for e, char in enumerate(character_list):
    character_dict[char] = e
time_numerical = [character_dict[char] for char in time_machine]

## One-hot representations

In [54]:
def one_hots(numerical_list, vocab_size=vocab_size):
    result = []
    for idx in numerical_list:
        array = nd.zeros(shape=(1, vocab_size), ctx=ctx)
        array[0, idx] = 1.0
        result.append(array)
    return result

In [55]:
def textify(vector_list):
    result = ""
    for vector in vector_list:
        vector = vector[0]
        result += character_list[int(nd.argmax(vector, axis=0).asscalar())]
    return result

## Preparing the data for training

In [56]:
seq_length = 64
dataset = [one_hots(time_numerical[i*seq_length:(i+1)*seq_length]) for i in range(int(np.floor((len(time_numerical)-1)/seq_length)))]
batch_size = 32
sequences_per_batch_row = int(np.floor(len(dataset))/batch_size)
data_rows = [dataset[i*sequences_per_batch_row:i*sequences_per_batch_row+sequences_per_batch_row] 
            for i in range(batch_size)]

In [57]:
def stack_the_datasets(datasets):
    full_dataset = []
    # iterate over the sequences
    for s in range(len(datasets[0])):
        sequence = []
        # iterate over the elements of the seqeunce
        for elem in range(len(datasets[0][0])):
            sequence.append(nd.concatenate([ds[s][elem].reshape((1,-1)) for ds in datasets], axis=0))
        full_dataset.append(sequence)
    return(full_dataset)     

In [58]:
training_data = stack_the_datasets(data_rows)

## Preparing our labels

In [59]:
labels = [one_hots(time_numerical[i*seq_length+1:(i+1)*seq_length+1]) for i in range(int(np.floor((len(time_numerical)-1)/seq_length)))]
label_rows = [labels[i*sequences_per_batch_row:i*sequences_per_batch_row+sequences_per_batch_row] for i in range(batch_size)]
training_labels = stack_the_datasets(label_rows)

## Long short-term memory (LSTM) RNNs

[Placeholder for explanation]

$$g_t = \text{tanh}(X_t W_{xg} + h_{t-1} W_{hg} + b_g)$$
$$i_t = \sigma(X_t W_{xi} + h_{t-1} W_{hi} + b_i)$$
$$f_t = \sigma(X_t W_{xf} + h_{t-1} W_{hf} + b_f)$$
$$o_t = \sigma(X_t W_{xo} + h_{t-1} W_{ho} + b_o)$$
$$c_t = f \odot c_{t-1} + g_t \odot i_t$$
$$h_t = \text{tanh}(c_t) \odot o_t $$

## Allocate parameters

In [60]:
num_inputs = 77
num_hidden = 256
num_outputs = 77

########################
#  Weights connecting the inputs to the hidden layer
########################
Wxg = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxi = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxf = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01
Wxo = nd.random_normal(shape=(num_inputs,num_hidden), ctx=ctx) * .01

########################
#  Recurrent weights connecting the hidden layer across time steps
########################
Whg = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Whi = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Whf = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01
Who = nd.random_normal(shape=(num_hidden,num_hidden), ctx=ctx)* .01

########################
#  Bias vector for hidden layer
########################
bg = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bi = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bf = nd.random_normal(shape=num_hidden, ctx=ctx) * .01
bo = nd.random_normal(shape=num_hidden, ctx=ctx) * .01


########################
# Weights to the output nodes
########################
Why = nd.random_normal(shape=(num_hidden,num_inputs), ctx=ctx) * .01
by = nd.random_normal(shape=num_inputs, ctx=ctx) * .01

## Attach the gradients

In [61]:
params = [Wxg, Wxi, Wxf, Wxo, Whg, Whi, Whf, Who, bg, bi, bf, bo]
params += [Why, by]

for param in params:
    param.attach_grad()
for param in params:
    param.attach_grad()

## Softmax Activation

In [62]:
def softmax(y_linear, temperature=1.0):
    lin = (y_linear-nd.max(y_linear)) / temperature
    exp = nd.exp(lin)
    partition =nd.sum(exp, axis=0, exclude=True).reshape((-1,1))
    return exp / partition

## Define the model

In [63]:
def lstm_rnn(inputs, h, c, temperature=1.0):
    outputs = []
    for X in inputs:
        g = nd.tanh(nd.dot(X, Wxg) + nd.dot(h, Whg) + bg)
        i = nd.sigmoid(nd.dot(X, Wxi) + nd.dot(h, Whi) + bi)
        f = nd.sigmoid(nd.dot(X, Wxf) + nd.dot(h, Whf) + bf)
        o = nd.sigmoid(nd.dot(X, Wxo) + nd.dot(h, Who) + bo)
        #######################
        #
        #######################
        c = f * c + g * i
        h = nd.tanh(c * o)
        #######################
        #
        #######################
        yhat_linear = nd.dot(h, Why) + by
        yhat = softmax(yhat_linear, temperature=temperature) 
        outputs.append(yhat)
    return (outputs, h, c)

## Cross-entropy loss function

In [64]:
def cross_entropy(yhat, y):
    return - nd.mean(nd.sum(y * nd.log(yhat), axis=0, exclude=True))

## Averaging the loss over the sequence

In [65]:
def average_ce_loss(outputs, labels):
    assert(len(outputs) == len(labels))
    total_loss = nd.array([0.], ctx=ctx)
    for (output, label) in zip(outputs,labels):
        total_loss = total_loss + cross_entropy(output, label)
    return total_loss / len(outputs)

## Optimizer

In [66]:
def SGD(params, lr):    
    for param in params:
        param[:] = param - lr * param.grad

## Generating text by sampling

In [67]:
def sample(prefix, num_chars, temperature=1.0):
    #####################################
    # Initialize the string that we'll return to the supplied prefix
    #####################################
    string = prefix

    #####################################
    # Prepare the prefix as a sequence of one-hots for ingestion by RNN
    #####################################
    prefix_numerical = [character_dict[char] for char in prefix]
    input = one_hots(prefix_numerical)
    
    #####################################
    # Set the initial state of the hidden representation ($h_0$) to the zero vector
    #####################################    
    h = nd.zeros(shape=(1, num_hidden), ctx=ctx)
    c = nd.zeros(shape=(1, num_hidden), ctx=ctx)

    #####################################
    # For num_chars iterations,
    #     1) feed in the current input
    #     2) sample next character from from output distribution
    #     3) add sampled character to the decoded string
    #     4) prepare the sampled character as a one_hot (to be the next input)
    #####################################    
    for i in range(num_chars):
        outputs, h, c = lstm_rnn(input, h, c, temperature=temperature)
        choice = np.random.choice(77, p=outputs[-1][0].asnumpy())
        string += character_list[choice]
        input = one_hots([choice])
    return string

In [ ]:
epochs = 5
moving_loss = 0.

learning_rate = 2.0

# state = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
for e in range(epochs):
    ############################
    # Attenuate the learning rate by a factor of 2 every 100 epochs.
    ############################
    if ((e+1) % 100 == 0):
        learning_rate = learning_rate / 2.0
    h = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
    c = nd.zeros(shape=(batch_size, num_hidden), ctx=ctx)
    for i, (data, label) in enumerate(zip(training_data, training_labels)):
        data_one_hot = data
        label_one_hot = label
        with autograd.record():
            outputs, h, c = lstm_rnn(data_one_hot, h, c)
            loss = average_ce_loss(outputs, label_one_hot)
            loss.backward()
        SGD(params, learning_rate)

        ##########################
        #  Keep a moving average of the losses
        ##########################
        if (i == 0) and (e == 0):
            moving_loss = nd.mean(loss).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(loss).asscalar()
      
    print("Epoch %s. Loss: %s" % (e, moving_loss)) 
#     print(sample("The Time Ma", 1024, temperature=.1))
#     print(sample("The Medical Man rose, came to the lamp,", 1024, temperature=.1))
            

Epoch 0. Loss: 2.26372341333
The Time Mas and the the sas the the the the the the the the the the the sall the the sore the the the sith the she the sar the the the sas the sore the the the sar the sas the sith the the sore the sas the the sar the the the sar the the the sith the the the the the me the sore the sas the sas the the the the the she the sore the sar the the sore the sas the sith the sas the the the sore and the the the sare the sas the the the the the the the sore the sar the sore the sall the sore and the the the the the the sound the sas the sore the the sore the the the the the sore the sas the she the the the sas the sas the the the the sall the sall the sar the sar the the sore the sall the she the sore the sare and the the the sar the the the sore the the sall the sith the sore the the she the sall the sar the sas the the sith the sore the sas the sith the sore the sas the sith the sare and the sore the the sas the sar the sound the sas the sore the the sore the sas

Epoch 4. Loss: 2.10364193748
The Time Mas the shere sore and the sand the sand the shere and the sill the shere and the shere and the sind the sall and the sind the shere the shere and the sall the sare and the sare and the shere of the sind the sind the shere and the shere and the shad the shere of the sind the shere shere was the sind and the sind the seren the shere and the shere and the sare and the shere and the sind the shere the shere and the shere and the sand the sall the shere and the sare and the sind the shere sore the sind the shere and the sand the shere was the sind the sare and the shere of the sind the sand the sand the sare and the shere and the sind the sind the sand the shere and the sare and the shere and the shere and the shere and the shere and the sind the store the sind the shere and the shere and the sind the sare and the sime and the shere the shere and the sare and the shere and the shere and the shere shere and the shat the sare and the sind the shere and t

Epoch 8. Loss: 1.98947375763
The Time Mar of the sead the sand the sand the sand the sand the shere and the sand the sime and the sand the sand the same and the sind the same and the sand the same and the sand the sand the sime of the sand the sould the sand the sain the same and the sound the sand the sain the same and the sind the same and the sand the sand the sand the same and the sind the sand the sime and the sand the same of the seare and the sand the some and the singer and the sand the sand the sand the sand the same and the sand the sind and shere was and the sand the sand the sand the sand the sand the same and the sime and the sand the sare and the sand the sand the sand the same of the sand the sand the same and the sime and the sinder and the sand the sand the same and the sand the same and the sand the sand the sand the sand the sand the same and the sand the sime of the sime and the sand the same and the sime and the sinder the same and the sind the sand the same the sa

Epoch 12. Loss: 1.90328281295
The Time Mast of the same and the sain the same of the sain the sain the sain the sain the sain the seare of the sain the same and so for a shill so the sain the sain the same of the sain the sain the sume and the sain the sain the same and the sume and the sume and the sain the sain the sain the some of the sain the same and the sain the same and the sain the sain the sain the same and the sain the sain the some of the sain the sain the sain the sain the sain the sain the sain the sain the sain the sain the same and the sain the some of the see and the sain the sain the sain the sain the sain the same and the sain the same and the sain the same and the sain the sain the same and the sain the same and the sain the sain the sain the sain the same and the sain the sain the same of the sain the sain the sain the same and the sain the some of the sain the same and so for a sumpent of the sain the some and the sain the sain the sain the some of the sain the sai

Epoch 16. Loss: 1.83328013166
The Time Machine of the stranged to the stranged to the strong the strange of the strange of the strange of the strong the sain the some of the stread the strange of the strange of the stread the stranged the same and the streat of the strange of the sain the some of the strange of the stread and the stone of the stread and sow and the strange of the stranged and the stranged and the stranged to me the sain the same and the sain the strange of the sain the sain the some of the stread and the strange of the strange of the strange of the strange of the strange of the stranged and the sain the some of the sain the stone of the sain of the stread and the strange of the stread the said the sain the some of the sain the some and the said the sain the stranged and the strange of the stread and the streat of the stranged the said the strange of the stread and the sain the store a store a store of the sain the strange of the stranged and the said the sain the stran

...

Epoch 992. Loss: 0.146372802595
The Time Machine, a vast lent of the mature her eided in the sid. I fangied in flacture of a stand about me, and along the same beas
fright.
When extrem how a
forned into a fifult out of the ruins. I felt the intence, back foom the fact with a slitther and must feel fares, pouredly that
selion the peaps of a matume
fright.

'As they were they for flany against the old arme
of what my hand, like any curtoed besors, came upon I faint towards a will be as well-world bright red slemping this weym. Then I slept, and white, any meder I back to silvoky chel redmst
ccals, the faint
dew, pomarant! I seates hop to him into the fustred me a delichoos. He seemed to surproseef men a quityelle. Silenestly, and that the sun wonder on. Time starkind the flame. But I pared sce, to,
that slewping the passes in but there was a most, and it was a stirf, and in a hoilentle duefes. The thing I had
seen at first inceppocation a long gallery dark, to the sun
had changed the app

Epoch 996. Loss: 0.147315925309
The Time Machine, a nother and expluined. The freeq came van seemed to be conveniented, as immine
again, the great paindly under the strentth or some liker and each faint at first incompontalital, and transt under the surfine's was still over the whole postibld. At first I saw a nown the hillt lifect and present reallecticg--tan that I was to see the lever previty of the darkness
continged me.

'I amoke a first small wayed becare; Ins at the fill, world, in seesck, paist becamen in my matches of a freed of
their marness of the palace. I thought I went of the gallery was of the distant.
At fers, the glace of the world of wait, but more indeless to telling possably triught some our destig blacks of bloms
vegative
dradding swinch-tove in the
darknest timee aster as I turned, and from the and lengurest of the chied liker snuw would still get over the holitopred; the
new candless was a recolder and sway the sound of buildist down some white and the cashing th

Epoch 1000. Loss: 0.148941815207
The Time Machine, a vase, at first incontinently and the white glads
greature, were into the sun.'

'Ifoning upof all the accountnes
beaus. The sun had come to see that one of the little people as they closen. The moon was suelt approached upon muservalions. They caugh the
reckion asome the who soff cider, linis was could exerthined and blockese came on
the descle ane of maky should with reclinged by an exploss,
and he was thinking of me the filers of the fear of stopping, came back. I sheoped once of the darkness of the
geses of the alratation from of silvend upon the Time Machine. I had so miked anoment.'





IILone_ upof himb kut be in that my match who
eads a dazl rain-smaghtels has on a lownest dark, and the moon was seena rushes all ghisted and once fear the model ruthed to be stome. I began to reall it was confinentured flowers, abone the might I could been saw how fan again
to tate, and so our
of account, the ground than the rein faces where as

Epoch 1004. Loss: 0.145379033695
The Time Machine, a nother and expluined. The freeq catess to any renclusent chanse of the little people a dreamed for an enement that her
proseed the lever, and I came to discover a feeling of
we were that the have
during the point of brok
speadity, and intelligence untouch.'

'The darkness greatfole alk, and into aboun
aminguiture,
seemed to me, was
absenunite
strange. I had alpeys, animals in the easy and no nengured in their acain, about the
Time Machine, and I did not read a miny after and
the night
before myself, I pancedded with persipe. 'Was tood up and I noticul that me as it was evidedating is the contailest and fail, the morning sunset me stood of a
moning in the back of the sun
had ceatained mere speed. I chapred against my now if I could no enge hother that species, as me in the
white barkly one, and I could see no sign of the ruing sway wher work of the little people was unnell, those was hullness grew freatherions to e
om the arms
had bee

Epoch 1008. Loss: 0.14549407388
The Time Machine, a nother and expluined, a vast mere creature of the macalurish, and his fect as it. I thought of the palace, which was
drowng time, and its a sweep and legst, and surprive in the darkness I
how it is some leaster by, and these morsing me worlowards it burst, and in the flick, and the whole world sove blotoun incredulling. I thought of the petulat, nor a mechadiallo.

'Sontumed attentuled frew the face of the opered with a
trangh. There were other ranger
hand an uniforteld in the fire. That I could need fren.. They wanted to the smoting--that drey sunstances of her starting before me. It lay I was oftelling, havidy in only a great weatheres of the geally glo,id baller wassitered. The thunstactive raigher destiting artson. All the end above to my heart and speed done still twildless before me. I went up the one for I came to the smoth--forther sofmer it moss was shadow. Then the floo was mers had would exeated. At suchadoned free nenseef 

## Conclusions

For whinges or inquiries, [open an issue on  GitHub.](https://github.com/zackchase/mxnet-the-straight-dope)